In [285]:
import pandas as pd
import spacy
import numpy as np
from collections import defaultdict

# Loading of Data
Here, we load a corpus from a sub-Reddit via a text file. Each line represents a new post or comment of a post.

In [286]:
# Load the data into a dataframe
df = pd.read_csv("../data/AskReddit_processed.txt", sep='\t', names=['Text'])

In [287]:
# Explore the dataframe
print(f"Dimension: {df.shape}")
df.head()

Dimension: (12859, 1)


,Text
0,What is your creepy/paranormal experience that...
1,What is something you have witnessed but no on...
2,what movies do you think have the craziest bac...
3,What is something you do that you know is bad ...
4,for me it's smelling sharpies lol


# Data Preparation

Before we can perform Part-of-Speech (POS) tagging via Hidden Markov Model (HMM) and running Viterbi algorithm,
we will first require an annotated dataset. This can be achieved via the corpus which we loaded before this,
before we can use it to create a HMM model to perform POS tagging via Viterbi algorithm.

## Annotating the Dataset

Annotating the dataset refer to assigning each word its corresponding POS tag in the corpus that is used to train
our HMM model. For the purpose of this demonstration of POS tagging via HMM and Viterbi, we will make use of SpaCy
to annotate our dataset.

However, do note that in practice, annotation might be better done by experts who are able
to accurately tag each word in the corpus with its correct POS tag.

> Note: Install en_core_web_sm first before running this cell by running:
> `python -m spacy download en_core_web_sm`

In [288]:
# Note: Install en_core_web_sm first before running this cell
#       by doing: python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_sm")

def pos_tag(sentence):
    # Process the sentence using the spaCy NLP pipeline
    doc = nlp(sentence)
    # Extract tokens and their POS tags
    tagged_tokens = [(token.text.lower(), token.pos_) for token in doc]
    return tagged_tokens

Now we annotate our corpus using SpaCy by applying the function `pos_tag` which we wrote.

In [289]:
# Apply Spacy POS tagging on data and save in the column name "POS_Tags"
df['POS_Tags'] = df['Text'].apply(pos_tag)

# Explore data after SpaCy POS tagging
df.head()

,Text,POS_Tags
0,What is your creepy/paranormal experience that...,"[(what, PRON), (is, AUX), (your, PRON), (creep..."
1,What is something you have witnessed but no on...,"[(what, PRON), (is, AUX), (something, PRON), (..."
2,what movies do you think have the craziest bac...,"[(what, PRON), (movies, NOUN), (do, AUX), (you..."
3,What is something you do that you know is bad ...,"[(what, PRON), (is, AUX), (something, PRON), (..."
4,for me it's smelling sharpies lol,"[(for, ADP), (me, PRON), (it, PRON), ('s, AUX)..."


## Preparing Data for Training

Next, we initialise a list of words with its corresponding POS tags for ease of training in the later steps (to
calculate probabilities).

In [312]:
# Flattens POS_Tags of train_df into a single list of tuples
flattened_list = [tup for sublist in df['POS_Tags'] for tup in sublist]
flattened_list

[('what', 'PRON'),
 ('is', 'AUX'),
 ('your', 'PRON'),
 ('creepy', 'ADJ'),
 ('/', 'SYM'),
 ('paranormal', 'ADJ'),
 ('experience', 'NOUN'),
 ('that', 'PRON'),
 ('you', 'PRON'),
 ('ca', 'AUX'),
 ('n’t', 'PART'),
 ('explain', 'VERB'),
 ('?', 'PUNCT'),
 ('what', 'PRON'),
 ('is', 'AUX'),
 ('something', 'PRON'),
 ('you', 'PRON'),
 ('have', 'AUX'),
 ('witnessed', 'VERB'),
 ('but', 'CCONJ'),
 ('no', 'DET'),
 ('one', 'NOUN'),
 ('will', 'AUX'),
 ('believe', 'VERB'),
 ('?', 'PUNCT'),
 ('what', 'PRON'),
 ('movies', 'NOUN'),
 ('do', 'AUX'),
 ('you', 'PRON'),
 ('think', 'VERB'),
 ('have', 'VERB'),
 ('the', 'DET'),
 ('craziest', 'ADJ'),
 ('backstories', 'NOUN'),
 ('?', 'PUNCT'),
 ('what', 'PRON'),
 ('is', 'AUX'),
 ('something', 'PRON'),
 ('you', 'PRON'),
 ('do', 'VERB'),
 ('that', 'PRON'),
 ('you', 'PRON'),
 ('know', 'VERB'),
 ('is', 'AUX'),
 ('bad', 'ADJ'),
 ('for', 'ADP'),
 ('you', 'PRON'),
 ('?', 'PUNCT'),
 ('for', 'ADP'),
 ('me', 'PRON'),
 ('it', 'PRON'),
 ("'s", 'AUX'),
 ('smelling', 'VERB'),
 ('

Moreover, we also update our dataframe to have 2 new columns, `Words` and `Tags` which contains the words and
tags separately for ease of training in the later steps (calculating the probabilities).

In [291]:
# Extract words and tags from the tuples in the 'POS_Tags' column
df['Words'] = df['POS_Tags'].apply(lambda x: [pair[0] for pair in x])
df['Tags'] = df['POS_Tags'].apply(lambda x: [pair[1] for pair in x])
df['Tag_Init'] = df['Tags'].apply(lambda x: x[0])

# Explore the updated dataframe
df.head()

,Text,POS_Tags,Words,Tags,Tag_Init
0,What is your creepy/paranormal experience that...,"[(what, PRON), (is, AUX), (your, PRON), (creep...","[what, is, your, creepy, /, paranormal, experi...","[PRON, AUX, PRON, ADJ, SYM, ADJ, NOUN, PRON, P...",PRON
1,What is something you have witnessed but no on...,"[(what, PRON), (is, AUX), (something, PRON), (...","[what, is, something, you, have, witnessed, bu...","[PRON, AUX, PRON, PRON, AUX, VERB, CCONJ, DET,...",PRON
2,what movies do you think have the craziest bac...,"[(what, PRON), (movies, NOUN), (do, AUX), (you...","[what, movies, do, you, think, have, the, craz...","[PRON, NOUN, AUX, PRON, VERB, VERB, DET, ADJ, ...",PRON
3,What is something you do that you know is bad ...,"[(what, PRON), (is, AUX), (something, PRON), (...","[what, is, something, you, do, that, you, know...","[PRON, AUX, PRON, PRON, VERB, PRON, PRON, VERB...",PRON
4,for me it's smelling sharpies lol,"[(for, ADP), (me, PRON), (it, PRON), ('s, AUX)...","[for, me, it, 's, smelling, sharpies, lol]","[ADP, PRON, PRON, AUX, VERB, NOUN, NOUN]",ADP


Finally, we also prepare 2 sets, one containing all unique words in the corpus, and another all the unique tags.

In [292]:
# Create a set of all unique words and tags
all_words = set(word for word_list in df['Words'] for word in word_list)
all_tags = set(tag for tag_list in df['Tags'] for tag in tag_list)

# Model Learning

In this section, we attempt to calculate the start, transition and emission probabilities based on the prepared data
from the previous section. These probabilities will then be put together into matrices, called the initial state probability matrix, $\pi$, transition probability matrix, $A$, and emission probability matrix, $B$, respectively.

These matrices which contain the probabilities are necessary for us to perform POS tagging on a new sentence
via Viterbi algorithm.

## Initial State Probability

The initial state probability is the probability of a sequence starting in state $s_i$.

More precisely, $P(q_1 = s_i)=\frac{Count(<S>s_i)}{Count(<S>)}$, where $<S>s_i = $ # of sentences starting with state $s_i$, $<S> = $ # of sentences.

In [293]:
# Calculate the total number of unique tags and words
t_len = len(all_tags)
w_len = len(all_words)

# Get a list of unique tags
unique_tag_list = list(all_tags)

# Get a list of unique words
unique_word_list = list(all_words)

First, we initialise an empty initial state probability matrix $\pi$ of size 1 x `len(all_tags)`.

In [294]:
# Create an initial state probability matrix of dimension 1 x len(all_tags)
pi = np.zeros((1, t_len))

for i, tag in enumerate(unique_tag_list):
    pi[0, i] = np.sum(df["Tag_Init"] == tag)

pi /= len(df)

# Convert pi to a dataframe for better readability
pi_df = pd.DataFrame(pi, columns = unique_tag_list)
pi_df

,NUM,DET,INTJ,ADP,SPACE,CCONJ,PART,SCONJ,VERB,NOUN,PUNCT,SYM,AUX,ADJ,X,PROPN,PRON,ADV
0,0.016487,0.081033,0.038494,0.020764,0.000156,0.006299,0.008554,0.053348,0.112995,0.108951,0.006921,0.0007,0.028229,0.055525,0.005288,0.115094,0.291625,0.049537


## Emission Probability

The emission probability is the probability of POS tag $s_i$ generating a particular word $w_k$ at any time $t$.

More precisely, $P(o_t=w_k | q_t=s_i)=\frac{Count(w_k, s_i)}{Count(s_i)}$, where $o_t=$ observed state at time $t$,
$w_k=$ word $k$, $q_t=$ hidden state at time $t$, $s_i=$ POS tag $i$.

First, we initialise an empty emission probability matrix $B$ of size `len(all_tags)` x `len(all_words)`

In [295]:
# Create an emission probability matrix of dimension len(all_tags) x len(all_words)
# Compute P(w_k | s_i) and storing in a T x V matrix
B = np.zeros((t_len, w_len))

# def calculate_emission_prob(word, tag, train_data):
#     tag_list = [(t_word, t_tag) for t_word, t_tag in train_data if t_tag == tag]
#     count_tag = len(tag_list)
#     w_given_tag_list = [t_word for t_word, _ in tag_list if t_word == word]
#     count_w_given_tag = len(w_given_tag_list)
    
#     return count_w_given_tag / count_tag

tag_count = defaultdict(int)
w_given_tag_count = defaultdict(int)

for word, tag in flattened_list:
    # Store tag counts in tag_count dictionary
    tag_count[tag] += 1
    
    # Store word given tag count in w_given_tag_count dictionary
    w_given_tag_count[(word, tag)] += 1

for i, tag in enumerate(unique_tag_list):
    n_tag_occurrences = tag_count[tag]
    for j, word in enumerate(unique_word_list):
        count_w_given_tag = w_given_tag_count[(word, tag)]
        B[i, j] = count_w_given_tag / n_tag_occurrences

# Convert B to a dataframe for better readability
B_df = pd.DataFrame(B, columns=unique_word_list, index=unique_tag_list)
B_df

,disrespect,tfmr,dried,hath,9,grounded,mattress,immigrant,removing,gouger,...,unlike,whispered,foreseeable,combustion,teens,flex,level,caulken,time)and,cusack
NUM,0.000000,0.00000,0.000000,0.000000,0.006356,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DET,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTJ,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ADP,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SPACE,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CCONJ,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PART,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SCONJ,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VERB,0.000025,0.00000,0.000127,0.000025,0.000000,0.000000,0.000000,0.000000,0.000076,0.000000,...,0.000000,0.000025,0.000000,0.000000,0.000000,0.000000,0.000051,0.000025,0.000000,0.000000
NOUN,0.000021,0.00000,0.000000,0.000000,0.000000,0.000000,0.000041,0.000062,0.000000,0.000021,...,0.000000,0.000000,0.000000,0.000062,0.000103,0.000021,0.000965,0.000000,0.000021,0.000000


## Transition Probability

The transition probability is the probability of transitioning from state $s_i$ to state $s_j$ at any time $t$.

More precisely, $P(q_{t+1}=s_j | q_t=s_i)=\frac{Count(s_i s_j)}{Count(s_i)}$, where $q_t=$ hidden state at time $t$,
$s_i=$ POS tag $i$.

> Note that $P(tag_i | start-tag) = P(tag_i | '.')$

> Note that this function takes in a list of words and its corresponding POS tags to determine the emission probability
> for each word and tag.

In [296]:
def calcualte_transition_prob(t2, t1, train_data):
    tags = [t_tag for _, t_tag in train_data]
    count_t1 = len([t for t in tags if t == t1])
    count_t2_t1 = 0

    for i in range(len(tags) - 1):
        if tags[i] != t1 or tags[i + 1] != t2:
            continue
        count_t2_t1 += 1

    return count_t2_t1 / count_t1

Now, we create the transition probability matrix $A$ which has a dimension of `t_len` x `t_len`, where each column
is the new state (POS tag) that we want to transition to, and each row represent the old state (POS tag).

For example, $A[i, j]$ represents $P(tag_j | tag_i)$.

In [297]:
A = np.zeros((t_len, t_len))
for i, t1 in enumerate(list(all_tags)):
    for j, t2 in enumerate(list(all_tags)):
        A[i, j] = calcualte_transition_prob(t2, t1, flattened_list)

# Convert A to a dataframe for better readability
A_df = pd.DataFrame(A, columns = list(all_tags), index=list(all_tags))
A_df

,NUM,DET,INTJ,ADP,SPACE,CCONJ,PART,SCONJ,VERB,NOUN,PUNCT,SYM,AUX,ADJ,X,PROPN,PRON,ADV
NUM,0.031528,0.013476,0.000763,0.103738,0.000000,0.036868,0.012713,0.009916,0.016527,0.454361,0.151284,0.026443,0.022629,0.042715,0.002543,0.026697,0.031274,0.016527
DET,0.016139,0.014672,0.000051,0.000253,0.000101,0.000051,0.000051,0.000202,0.010422,0.629869,0.006172,0.001619,0.000202,0.247951,0.000101,0.048062,0.004300,0.019781
INTJ,0.027592,0.037625,0.065217,0.017559,0.000836,0.030100,0.003344,0.023411,0.050167,0.045987,0.436455,0.004181,0.014214,0.020903,0.000836,0.019231,0.172241,0.030100
ADP,0.030083,0.269214,0.002160,0.047481,0.000275,0.009818,0.004202,0.013588,0.039626,0.175745,0.036288,0.002788,0.006323,0.053686,0.000785,0.049366,0.236500,0.022071
SPACE,0.000000,0.046875,0.023438,0.140625,0.000000,0.007812,0.015625,0.093750,0.046875,0.085938,0.054688,0.000000,0.039062,0.015625,0.000000,0.281250,0.117188,0.031250
CCONJ,0.012287,0.075368,0.003967,0.021672,0.000193,0.001258,0.015577,0.038313,0.214977,0.116099,0.007643,0.001354,0.049536,0.059598,0.000387,0.030283,0.257353,0.094137
PART,0.003449,0.022416,0.000862,0.011639,0.000216,0.002371,0.009592,0.006035,0.691238,0.038043,0.020800,0.000323,0.076840,0.038043,0.000000,0.006574,0.014657,0.056903
SCONJ,0.004927,0.068163,0.000684,0.018752,0.000000,0.003422,0.015604,0.015467,0.029565,0.041473,0.018478,0.000411,0.042705,0.037777,0.000137,0.015877,0.661100,0.025459
VERB,0.013291,0.133240,0.002866,0.192746,0.000076,0.020647,0.065517,0.033101,0.024680,0.082460,0.075916,0.001826,0.007153,0.048928,0.000380,0.014788,0.227166,0.055219
NOUN,0.004807,0.013988,0.002259,0.173096,0.000185,0.080478,0.025388,0.031386,0.064148,0.109728,0.285884,0.004293,0.060369,0.014666,0.001027,0.012653,0.073782,0.041862


# Viterbi Algorithm

Now that we have already prepared our initial state matrix, $\pi$, transition probability matrix, $A$ and our emission
probability matrix, $B$, we have our model ready to be used with Viterbi algorithm, which uses dynamic programming, to
perform POS tagging based on $\pi$, $A$ and $B$.

In [315]:
def viterbi(words, pos_tags, A, B, pi, default_prob=1e-5):
    T = len(words)
    N = len(pos_tags)
    
    # Initialize the Viterbi matrix and backtrack matrix
    viterbi_matrix = np.zeros((N, T))
    backtrack_matrix = np.zeros((N, T), dtype=int)
    
    # Initialisation step
    for s, pos_tag in enumerate(pos_tags):
        viterbi_matrix[s, 0] = pi.loc[0, pos_tag] * (B.loc[pos_tag, words[0]] if words[0] in B.columns else default_prob)
        backtrack_matrix[s, 0] = 0
    
    # Recursion step
    for t in range(1, T):
        for s, pos_tag in enumerate(pos_tags):
            prob_transitions = viterbi_matrix[:, t-1] * A.loc[:, pos_tag]
            max_prob = np.max(prob_transitions)
            observed_emission_prob = B.loc[pos_tag, words[t]] if words[t] in B.columns else default_prob
            viterbi_matrix[s, t] = max_prob * observed_emission_prob
            backtrack_matrix[s, t] = np.argmax(prob_transitions)
    
    # Termination step
    best_path_prob = np.max(viterbi_matrix[:, -1])
    best_last_state = np.argmax(viterbi_matrix[:, -1])
    best_path = [pos_tags[best_last_state]]
    
    # Backtracking to find the best path
    for t in range(T-1, 0, -1):
        best_last_state = backtrack_matrix[best_last_state, t]
        best_path.insert(0, pos_tags[best_last_state])
    
    return best_path, best_path_prob

# Perform POS Tagging

Let's have a few test examples, and perform POS tagging on each of these sequences of words via our Viterbi algorithm
trained with our corpus!

In [323]:
# Test examples
test_cases = [
    "How are you doing?",
    "In Singapore, the weather is very hot!",
    "What do you mean by that?",
    "How do you tell between a cat and a dog?",
    "HS2914 is very interesting and I love it!"
]

for sentence in test_cases:
    # Normalise sentence
    sentence_norm = sentence.lower()
    doc = nlp(sentence_norm)

    # Tokenise sentence
    tokens = [token.text for token in doc]

    # Run viterbi on the tokens
    pos_tags, probability = viterbi(tokens, unique_tag_list, A_df, B_df, pi_df)

    print("===")
    print(f"Sentence: {sentence}")
    print(list(zip(tokens, pos_tags)))
    print(f"Probability: {probability}")

===
Sentence: How are you doing?
[('how', 'SCONJ'), ('are', 'AUX'), ('you', 'PRON'), ('doing', 'VERB'), ('?', 'PUNCT')]
Probability: 6.336674775742297e-13
===
Sentence: In Singapore, the weather is very hot!
[('in', 'ADP'), ('singapore', 'NOUN'), (',', 'PUNCT'), ('the', 'DET'), ('weather', 'NOUN'), ('is', 'AUX'), ('very', 'ADV'), ('hot', 'ADJ'), ('!', 'PUNCT')]
Probability: 3.9933143519406957e-26
===
Sentence: What do you mean by that?
[('what', 'PRON'), ('do', 'AUX'), ('you', 'PRON'), ('mean', 'VERB'), ('by', 'ADP'), ('that', 'PRON'), ('?', 'PUNCT')]
Probability: 1.993599052426198e-16
===
Sentence: How do you tell between a cat and a dog?
[('how', 'SCONJ'), ('do', 'AUX'), ('you', 'PRON'), ('tell', 'VERB'), ('between', 'ADP'), ('a', 'DET'), ('cat', 'NOUN'), ('and', 'CCONJ'), ('a', 'DET'), ('dog', 'NOUN'), ('?', 'PUNCT')]
Probability: 1.1664686811587063e-25
===
Sentence: HS2914 is very interesting and I love it!
[('hs2914', 'PRON'), ('is', 'AUX'), ('very', 'ADV'), ('interesting', 'ADJ')